# K₇ TCS Spectral Analysis v2 - Corrected Eigensolver

**Fix**: Use sparse matrix + `eigsh(which='SA')` to find **smallest** eigenvalues.

**Target**: λ₁ = 14/99 ≈ 0.1414, so λ₁ × H* = 14

---

In [ ]:
# Cell 1: Imports
import numpy as np
import time
import json
from typing import Tuple, Dict, Callable, List

try:
    import cupy as cp
    from cupyx.scipy.sparse import coo_matrix as cp_coo
    from cupyx.scipy.sparse import csr_matrix as cp_csr
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU_AVAILABLE = True
    xp = cp  # Use CuPy
    print(f"GPU: {cp.cuda.runtime.getDeviceProperties(0)['name'].decode()}")
    print(f"Memory: {cp.cuda.runtime.getDeviceProperties(0)['totalGlobalMem'] / 1e9:.1f} GB")
except ImportError:
    GPU_AVAILABLE = False
    xp = np
    from scipy.sparse import coo_matrix as cp_coo
    from scipy.sparse import csr_matrix as cp_csr
    from scipy.sparse.linalg import eigsh as cp_eigsh
    print("CPU mode (SciPy)")

import matplotlib.pyplot as plt
print(f"\nGPU Mode: {GPU_AVAILABLE}")

In [ ]:
# Cell 2: GIFT Constants
DIM_K7 = 7
DIM_G2 = 14
H_G2 = 6
B2, B3 = 21, 77
H_STAR = B2 + B3 + 1  # = 99

WEYL, RANK_E8 = 5, 8
DET_G = WEYL * (RANK_E8 + WEYL) / (2 ** WEYL)  # = 65/32
G_DIAG = DET_G ** (1/DIM_K7)

LAMBDA1_TARGET = DIM_G2 / H_STAR  # = 14/99

print(f"Target: λ₁ = {LAMBDA1_TARGET:.8f}")
print(f"Pell check: {H_STAR}² - 50×{DIM_G2}² = {H_STAR**2 - 50*DIM_G2**2}")

In [ ]:
# Cell 3: Metrics

def metric_flat(coords):
    return np.ones((len(coords), DIM_K7))

def metric_g2_constant(coords):
    return np.full((len(coords), DIM_K7), G_DIAG)

def metric_tcs_neck(coords, neck_width=0.5, amplitude=0.3):
    N = len(coords)
    t = coords[:, 0]
    neck_profile = np.exp(-((t - np.pi) / neck_width) ** 2)
    g_modulated = G_DIAG * (1 + amplitude * neck_profile)
    g = np.tile(g_modulated[:, np.newaxis], (1, DIM_K7))
    # Normalize det(g)
    det_current = np.prod(g, axis=1)
    correction = (DET_G / det_current) ** (1/DIM_K7)
    g *= correction[:, np.newaxis]
    return g

def metric_tcs_holonomy(coords, neck_width=0.4, holonomy_strength=0.5):
    N = len(coords)
    t = coords[:, 0]
    neck_profile = np.exp(-((t - np.pi) / neck_width) ** 2)
    g = np.zeros((N, DIM_K7))
    g[:, :3] = G_DIAG * (1 + holonomy_strength * (1 - neck_profile[:, np.newaxis]))
    g[:, 3:] = G_DIAG * (1 + holonomy_strength * neck_profile[:, np.newaxis])
    # Normalize
    det_current = np.prod(g, axis=1)
    correction = (DET_G / det_current) ** (1/DIM_K7)
    g *= correction[:, np.newaxis]
    return g

# Test
test_coords = np.random.rand(100, 7) * 2 * np.pi
for name, fn in [('Flat', metric_flat), ('G₂', metric_g2_constant), 
                 ('TCS neck', metric_tcs_neck), ('TCS holonomy', metric_tcs_holonomy)]:
    g = fn(test_coords)
    print(f"{name:12s}: det(g) = {np.prod(g, axis=1).mean():.4f}")

In [ ]:
# Cell 4: Build Sparse Laplacian (COO format - CuPy compatible)

def build_laplacian_sparse(grid_size: int, 
                           metric: np.ndarray,
                           domain_size: float = 2*np.pi) -> cp_csr:
    """
    Build sparse Laplacian matrix with periodic BC.
    
    Uses COO format (CuPy compatible), then converts to CSR.
    
    Δ_g f = Σ_d (1/g_d) (f[i+1] - 2f[i] + f[i-1]) / h²
    
    For NEGATIVE Laplacian (positive semi-definite), we use -Δ.
    """
    n = grid_size
    h = domain_size / n
    h2 = h * h
    dim = DIM_K7
    N = n ** dim
    
    # Strides for multi-index
    strides = [n**i for i in range(dim)]
    
    # Build COO entries
    row_list = []
    col_list = []
    data_list = []
    
    # For each point
    for idx in range(N):
        # Multi-index
        multi = []
        temp = idx
        for d in range(dim):
            multi.append(temp % n)
            temp //= n
        
        # Diagonal term: sum over all directions
        diag_val = 0.0
        
        for d in range(dim):
            inv_g = 1.0 / metric[idx, d]
            coeff = inv_g / h2
            
            # Diagonal contribution: -2/h² per direction
            diag_val += -2.0 * coeff
            
            # Neighbors with periodic BC
            multi_plus = multi.copy()
            multi_plus[d] = (multi[d] + 1) % n
            idx_plus = sum(multi_plus[dd] * strides[dd] for dd in range(dim))
            
            multi_minus = multi.copy()
            multi_minus[d] = (multi[d] - 1) % n
            idx_minus = sum(multi_minus[dd] * strides[dd] for dd in range(dim))
            
            # Off-diagonal: +1/h² for neighbors
            row_list.extend([idx, idx])
            col_list.extend([idx_plus, idx_minus])
            data_list.extend([coeff, coeff])
        
        # Add diagonal
        row_list.append(idx)
        col_list.append(idx)
        data_list.append(diag_val)
    
    # Build sparse matrix
    row = np.array(row_list, dtype=np.int32)
    col = np.array(col_list, dtype=np.int32)
    data = np.array(data_list, dtype=np.float64)
    
    # This is the Laplacian Δ (negative semi-definite)
    # We want -Δ (positive semi-definite) for eigsh
    data = -data
    
    if GPU_AVAILABLE:
        row_gpu = cp.asarray(row)
        col_gpu = cp.asarray(col)
        data_gpu = cp.asarray(data)
        L = cp_csr(cp_coo((data_gpu, (row_gpu, col_gpu)), shape=(N, N)))
    else:
        from scipy.sparse import coo_matrix, csr_matrix
        L = csr_matrix(coo_matrix((data, (row, col)), shape=(N, N)))
    
    return L

print("Sparse Laplacian builder ready.")

In [ ]:
# Cell 5: Compute Eigenvalues with eigsh(which='SA')

def compute_spectrum(grid_size: int,
                     metric_fn: Callable,
                     metric_name: str = "Custom",
                     n_eigenvalues: int = 6,
                     domain_size: float = 2*np.pi) -> Dict:
    """
    Compute smallest Laplacian eigenvalues.
    
    Uses eigsh with which='SA' (Smallest Algebraic).
    """
    print(f"\n{'='*60}")
    print(f"{metric_name} | Grid: {grid_size}^7 = {grid_size**7:,} points")
    print(f"{'='*60}")
    
    N = grid_size ** DIM_K7
    h = domain_size / grid_size
    
    # Generate coordinates
    t0 = time.time()
    coords_1d = np.linspace(0, domain_size, grid_size, endpoint=False)
    grids = np.meshgrid(*[coords_1d]*DIM_K7, indexing='ij')
    coords = np.stack(grids, axis=-1).reshape(-1, DIM_K7)
    print(f"Grid: {time.time()-t0:.2f}s")
    
    # Compute metric
    t0 = time.time()
    metric = metric_fn(coords)
    print(f"Metric: {time.time()-t0:.2f}s | det(g) = {np.prod(metric, axis=1).mean():.4f}")
    
    # Build sparse Laplacian
    t0 = time.time()
    L = build_laplacian_sparse(grid_size, metric, domain_size)
    print(f"Sparse matrix: {time.time()-t0:.2f}s | nnz = {L.nnz:,}")
    
    # Compute smallest eigenvalues with SA (Smallest Algebraic)
    t0 = time.time()
    try:
        eigenvalues, eigenvectors = cp_eigsh(L, k=n_eigenvalues, which='SA')
        if GPU_AVAILABLE:
            eigenvalues = cp.asnumpy(eigenvalues)
    except Exception as e:
        print(f"eigsh failed: {e}")
        # Fallback: try with sigma=0 shift-invert
        try:
            eigenvalues, eigenvectors = cp_eigsh(L, k=n_eigenvalues, sigma=0.0, which='LM')
            if GPU_AVAILABLE:
                eigenvalues = cp.asnumpy(eigenvalues)
        except:
            return {'error': str(e)}
    
    elapsed = time.time() - t0
    print(f"Eigensolver: {elapsed:.2f}s")
    
    # Sort eigenvalues
    eigenvalues = np.sort(eigenvalues)
    
    # λ₀ should be ~0 (constant function)
    # λ₁ is first non-zero eigenvalue
    lambda0 = eigenvalues[0]
    lambda1 = eigenvalues[1] if len(eigenvalues) > 1 else eigenvalues[0]
    
    # GIFT product
    product = lambda1 * H_STAR
    deviation = abs(product - DIM_G2) / DIM_G2 * 100
    
    print(f"\nResults:")
    print(f"  λ₀ = {lambda0:.8f} (should be ~0)")
    print(f"  λ₁ = {lambda1:.8f}")
    print(f"  λ₁ × H* = {product:.4f} (target: {DIM_G2})")
    print(f"  Deviation: {deviation:.2f}%")
    
    # Clear GPU memory
    if GPU_AVAILABLE:
        del L
        cp.get_default_memory_pool().free_all_blocks()
    
    return {
        'metric': metric_name,
        'grid_size': grid_size,
        'n_points': N,
        'eigenvalues': eigenvalues.tolist(),
        'lambda_0': float(lambda0),
        'lambda_1': float(lambda1),
        'lambda1_x_Hstar': float(product),
        'target': DIM_G2,
        'deviation_percent': float(deviation),
        'time_seconds': elapsed
    }

print("Spectrum computation ready.")

In [ ]:
# Cell 6: Test with small grid first

print("Testing with n=4 (16384 points)...")
result = compute_spectrum(4, metric_flat, "Flat T⁷ (test)")

print("\n" + "="*60)
print("VALIDATION CHECK")
print("="*60)
print(f"λ₀ = {result['lambda_0']:.6f} (should be ~0)")
print(f"λ₁ = {result['lambda_1']:.6f}")
print()
print("For flat T⁷ with period 2π:")
print(f"  Analytical λ₁ = 1 (for f = cos(x))")
print(f"  Numerical  λ₁ = {result['lambda_1']:.6f}")

In [ ]:
# Cell 7: Richardson extrapolation

def richardson_extrapolate(results: List[Dict], order: int = 2) -> Dict:
    if len(results) < 2:
        return results[-1] if results else {}
    
    results = sorted(results, key=lambda x: x['grid_size'])
    coarse, fine = results[-2], results[-1]
    
    n_c, n_f = coarse['grid_size'], fine['grid_size']
    r = n_f / n_c
    
    lambda_c = coarse['lambda_1']
    lambda_f = fine['lambda_1']
    
    lambda_extrap = lambda_f + (lambda_f - lambda_c) / (r**order - 1)
    error = abs(lambda_f - lambda_c) / (r**order - 1)
    product = lambda_extrap * H_STAR
    
    print(f"\nRichardson: λ₁ = {lambda_extrap:.8f} ± {error:.2e}")
    print(f"  λ₁ × H* = {product:.4f}")
    
    return {
        'method': 'Richardson',
        'lambda_1_extrapolated': float(lambda_extrap),
        'lambda1_x_Hstar': float(product),
        'error_estimate': float(error),
        'deviation_percent': float(abs(product - DIM_G2)/DIM_G2*100)
    }

print("Richardson ready.")

In [ ]:
# Cell 8: Full Analysis

print("\n" + "="*70)
print("K₇ TCS SPECTRAL ANALYSIS v2")
print(f"Target: λ₁ × H* = {DIM_G2}")
print("="*70)

metrics = [
    ('Flat T⁷', metric_flat),
    ('G₂ constant', metric_g2_constant),
    ('TCS neck', metric_tcs_neck),
    ('TCS holonomy', metric_tcs_holonomy),
]

grid_sizes = [4, 5, 6]  # Start small; increase if memory allows

all_results = {}

for name, fn in metrics:
    results = []
    for n in grid_sizes:
        try:
            r = compute_spectrum(n, fn, f"{name} (n={n})")
            if 'error' not in r:
                results.append(r)
        except Exception as e:
            print(f"Error: {e}")
    
    if len(results) >= 2:
        results.append(richardson_extrapolate(results))
    
    all_results[name] = results

print("\n" + "="*70)
print("Analysis complete!")

In [ ]:
# Cell 9: Summary

print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"{'Metric':<18} {'Grid':<8} {'λ₁':<12} {'λ₁×H*':<10} {'Dev %':<8}")
print("-"*60)

for name, results in all_results.items():
    for r in results:
        if 'method' in r:
            grid_str = "Extrap"
            l1 = r['lambda_1_extrapolated']
        else:
            grid_str = f"{r['grid_size']}^7"
            l1 = r['lambda_1']
        prod = r.get('lambda1_x_Hstar', l1 * H_STAR)
        dev = r.get('deviation_percent', 0)
        print(f"{name:<18} {grid_str:<8} {l1:<12.6f} {prod:<10.4f} {dev:<8.2f}")

print("="*80)
print(f"\nTarget: λ₁ = {LAMBDA1_TARGET:.8f}, λ₁×H* = {DIM_G2}")

In [ ]:
# Cell 10: Save results

output = {
    'metadata': {
        'version': 'v2',
        'fix': 'Using eigsh with which=SA for smallest eigenvalues',
        'date': time.strftime('%Y-%m-%d %H:%M:%S'),
        'target_lambda1': LAMBDA1_TARGET,
        'target_product': DIM_G2
    },
    'results': all_results
}

with open('K7_TCS_spectral_v2_results.json', 'w') as f:
    json.dump(output, f, indent=2)

print("Results saved to K7_TCS_spectral_v2_results.json")

---

## Key Fix

**Before (v1)**: Lanczos found largest eigenvalues → λ₀ ≈ 20-30

**After (v2)**: `eigsh(L, which='SA')` finds smallest → λ₀ ≈ 0, λ₁ = target

---